In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Airbnb data processing").getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/18 04:41:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
listings = spark.read.csv(
    "data/listings.csv.gz",
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE",
)


In [3]:
review_locations = listings.select(listings.review_scores_location)
review_locations.show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.62|
|                  4.54|
|                  4.36|
|                   4.5|
|                  4.37|
|                   4.5|
|                  4.53|
|                  4.52|
|                  4.49|
|                  4.94|
|                  4.82|
|                  4.92|
|                  4.14|
|                  4.78|
|                  4.43|
|                  4.73|
|                  NULL|
|                  NULL|
|                  4.48|
|                  4.75|
+----------------------+
only showing top 20 rows


In [4]:
listings.select(listings.review_scores_location).show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.62|
|                  4.54|
|                  4.36|
|                   4.5|
|                  4.37|
|                   4.5|
|                  4.53|
|                  4.52|
|                  4.49|
|                  4.94|
|                  4.82|
|                  4.92|
|                  4.14|
|                  4.78|
|                  4.43|
|                  4.73|
|                  NULL|
|                  NULL|
|                  4.48|
|                  4.75|
+----------------------+
only showing top 20 rows


In [5]:
high_score_listings = listings.filter(listings.review_scores_location > 4.5).select(
    "id", "price", "name", "review_scores_location"
)

high_score_listings.show(20, truncate=False)

+------+-------+------------------------------------------------+----------------------+
|id    |price  |name                                            |review_scores_location|
+------+-------+------------------------------------------------+----------------------+
|264776|$297.00|Huge Four Bedroom Apartment                     |4.62                  |
|264777|$98.00 |One Bedroom Apartment                           |4.54                  |
|264782|$120.00|One Bedroom Garden Apartment                    |4.53                  |
|264783|$216.00|Four Bedroom Garden Apartment                   |4.52                  |
|266037|$62.00 |Central London with Stunning Views!             |4.94                  |
|268398|$66.00 |Also five minutes to South Bank                 |4.82                  |
|270600|$73.00 |Patio Apartment in London (Twickenham)          |4.92                  |
|425143|NULL   |luxury 1bed in Chelsea Bridge Wharf             |4.78                  |
|426354|$200.00|1 bed

In [6]:
high_score_listings.dropna().show(20, truncate=False)

+------+-------+------------------------------------------------+----------------------+
|id    |price  |name                                            |review_scores_location|
+------+-------+------------------------------------------------+----------------------+
|264776|$297.00|Huge Four Bedroom Apartment                     |4.62                  |
|264777|$98.00 |One Bedroom Apartment                           |4.54                  |
|264782|$120.00|One Bedroom Garden Apartment                    |4.53                  |
|264783|$216.00|Four Bedroom Garden Apartment                   |4.52                  |
|266037|$62.00 |Central London with Stunning Views!             |4.94                  |
|268398|$66.00 |Also five minutes to South Bank                 |4.82                  |
|270600|$73.00 |Patio Apartment in London (Twickenham)          |4.92                  |
|426354|$200.00|1 bedroom flat with big balcony!                |4.73                  |
|427584|$129.00|Hackn

In [7]:
high_score_listings.schema["price"]

StructField('price', StringType(), True)

In [10]:
from pyspark.sql.functions import regexp_replace

price_num_df = listings.withColumn(
    "price_num", regexp_replace("price", "[$,]", "").cast("float")
)

price_num_df.schema["price_num"]

StructField('price_num', FloatType(), True)

In [11]:
price_num_df.select("price_num", "name").show(20, truncate=False)

+---------+--------------------------------------------------+
|price_num|name                                              |
+---------+--------------------------------------------------+
|297.0    |Huge Four Bedroom Apartment                       |
|98.0     |One Bedroom Apartment                             |
|148.0    |Two Bedroom Newly Refurbished Apartment           |
|144.0    |Refurbished Two Bedroom Apartment                 |
|157.0    |Spacious refurbished 2 bedroom apt with balcony   |
|148.0    |Two Bedrooms Garden Maisonette                    |
|120.0    |One Bedroom Garden Apartment                      |
|216.0    |Four Bedroom Garden Apartment                     |
|238.0    |Huge Three Bedroom Flat with parking and terrace  |
|62.0     |Central London with Stunning Views!               |
|66.0     |Also five minutes to South Bank                   |
|73.0     |Patio Apartment in London (Twickenham)            |
|NULL     |Heathrow BNB - Home Away From Home!         

In [12]:
price_num_df.filter(
    (price_num_df.price_num < 100) & (price_num_df.review_scores_location > 4.5)
).select("name", "price", "review_scores_location").show(truncate=False)

+-------------------------------------------------+------+----------------------+
|name                                             |price |review_scores_location|
+-------------------------------------------------+------+----------------------+
|One Bedroom Apartment                            |$98.00|4.54                  |
|Central London with Stunning Views!              |$62.00|4.94                  |
|Also five minutes to South Bank                  |$66.00|4.82                  |
|Patio Apartment in London (Twickenham)           |$73.00|4.92                  |
|Boutique Room w/ Private Bath, Balcony           |$70.00|4.88                  |
|Holiday London DB Room Let-on going              |$72.00|4.77                  |
|You are GUARANTEED to love this                  |$74.00|4.77                  |
|Bedroom In Great Location Stratford              |$52.00|4.57                  |
|Bright Double + workspace in spacious Garden Flat|$51.00|4.77                  |
|SPACIOUS ROOM I

In [14]:
price_num_df.filter("price_num < 100 AND review_scores_location > 4.5").select(
    "name", "price", "review_scores_location"
).show(truncate=False)


+-------------------------------------------------+------+----------------------+
|name                                             |price |review_scores_location|
+-------------------------------------------------+------+----------------------+
|One Bedroom Apartment                            |$98.00|4.54                  |
|Central London with Stunning Views!              |$62.00|4.94                  |
|Also five minutes to South Bank                  |$66.00|4.82                  |
|Patio Apartment in London (Twickenham)           |$73.00|4.92                  |
|Boutique Room w/ Private Bath, Balcony           |$70.00|4.88                  |
|Holiday London DB Room Let-on going              |$72.00|4.77                  |
|You are GUARANTEED to love this                  |$74.00|4.77                  |
|Bedroom In Great Location Stratford              |$52.00|4.57                  |
|Bright Double + workspace in spacious Garden Flat|$51.00|4.77                  |
|SPACIOUS ROOM I

In [15]:
listings.select(listings.property_type).distinct().show(truncate=False)

+----------------------------------+
|property_type                     |
+----------------------------------+
|Private room in lighthouse        |
|Private room in loft              |
|Private room in earthen home      |
|Entire chalet                     |
|Earthen home                      |
|Farm stay                         |
|Entire rental unit                |
|Shared room in hostel             |
|Shared room                       |
|Private room in condo             |
|Room in boutique hotel            |
|Private room in religious building|
|Room in bed and breakfast         |
|Private room in casa particular   |
|Private room in bungalow          |
|Entire cabin                      |
|Entire guesthouse                 |
|Hut                               |
|Private room in nature lodge      |
|Entire guest suite                |
+----------------------------------+
only showing top 20 rows


In [16]:
listings.select(listings.property_type, listings.room_type).distinct().show(
    truncate=False
)


+----------------------------------+---------------+
|property_type                     |room_type      |
+----------------------------------+---------------+
|Room in hostel                    |Hotel room     |
|Private room in casa particular   |Private room   |
|Dome                              |Entire home/apt|
|Entire serviced apartment         |Entire home/apt|
|Private room in loft              |Private room   |
|Shipping container                |Entire home/apt|
|Private room in villa             |Private room   |
|Farm stay                         |Entire home/apt|
|Room in hotel                     |Hotel room     |
|Shared room in rental unit        |Shared room    |
|Private room in guest suite       |Private room   |
|Room in rental unit               |Hotel room     |
|Room in serviced apartment        |Hotel room     |
|Private room in serviced apartment|Private room   |
|Private room in hostel            |Private room   |
|Shared room                       |Shared roo

In [17]:
listings.select(listings.property_type).distinct().write.csv("data/property_types")
